In [19]:
from transformers import (
    pipeline,
    PreTrainedModel,
    PreTrainedTokenizerFast,
    AutoTokenizer,
    AutoModelWithLMHead,
    AutoModelForSequenceClassification,
    GPT2LMHeadModel,
    PreTrainedTokenizerBase,
    PegasusTokenizer,
    PegasusForConditionalGeneration,
    RobertaForSequenceClassification)
generator = pipeline(
            "text-generation",
            model="gpt2",
            device=0)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [165]:
formatted_prompt = "Great Service. It was"

In [166]:
nah ... why bother when there are so many other options around .

SyntaxError: invalid syntax (96296406.py, line 1)

In [ ]:
manual_prompt = "Good place! The sentiment of this sentence is"

In [ ]:
manual_prompt_2 = "Great Service. The sentiment of this sentence is"

In [20]:
pos_verbalizer = '\u0120good' #\u0120wonderful'#'\u0120wonderful'#'\u0120positive'
neg_verbalizer = '\u0120bad'#'\u0120bad'#'\u0120bad'#'\u0120negative'

In [21]:
pos_id = generator.tokenizer.convert_tokens_to_ids(pos_verbalizer)
neg_id = generator.tokenizer.convert_tokens_to_ids(neg_verbalizer)

In [22]:
import torch
def evaluate_acc(formatted_prompt):
    encoding = generator.tokenizer(formatted_prompt,return_tensors='pt').to(0)
    output = generator.model(**encoding)
    output_probs = torch.nn.functional.softmax(output.logits[0,-1][[pos_id,neg_id]],dim=-1)
    if output_probs[0] < output_probs[1]:
        return 1
    else:
        return 0
        
    #print('positive:',output_probs[0])
    #print('negative:',output_probs[1])
#print(generator.tokenizer.convert_ids_to_tokens(pos_id), output_probs[pos_id])
#print(generator.tokenizer.convert_ids_to_tokens(neg_id), output_probs[neg_id])
    #values, indices = torch.topk(torch.nn.functional.softmax(output.logits[0,-1],dim=-1), k=10)
    
#for v, t in zip(values, generator.tokenizer.convert_ids_to_tokens(indices)):
#    print(t, '{:.3f}'.format(v), '{:.3f}'.format(v/sum(values)))

In [247]:
# test whole dataset
evaluate_acc(formatted_prompt)

positive: tensor(0.4118, device='cuda:0', grad_fn=<SelectBackward0>)


1

In [23]:
dataset_path = './data/yelp-gpt2-control-only/raw/sentiment.train.0'

In [24]:
with open(dataset_path) as f: 
    sentences_train_0 = [line.strip() for line in f]

In [206]:
sentences_train_0[0]+" ����������"

'i was sadly mistaken . ����������'

In [25]:
count = 0
for data in sentences_train_0[:1000]:
    formatted_prompt = data+" <UNK><UNK><UNK><UNK><UNK>"#" It was"#" ����������"
    acc = evaluate_acc(formatted_prompt)
    if acc == 1:
        count += 1

In [26]:
count/1000

0.331

# test

In [186]:
test_dataset_path = './data/yelp-gpt2-control-only/raw/sentiment.test.0'

In [187]:
with open(test_dataset_path) as f: 
    sentences_test_0 = [line.strip() for line in f]

In [14]:
count = 0
for data in sentences_test_0[:2000]:
    formatted_prompt = data+ " <UNK><UNK><UNK><UNK><UNK>"#" ����������"
    acc = evaluate_acc(formatted_prompt)
    if acc == 1:
        count += 1

NameError: name 'sentences_test_0' is not defined

In [ ]:
count/2000

In [33]:
sentences_train_1[0]

'excellent food .'

# positive-train

In [27]:
train_dataset_path_1 = './data/yelp-gpt2-control-only/raw/sentiment.train.1'
with open(train_dataset_path_1) as f: 
    sentences_train_1 = [line.strip() for line in f]

In [28]:
len(sentences_train_1)+len(sentences_train_0)

444101

In [29]:
count = 0
for data in sentences_train_1[:1000]:
    formatted_prompt = data+" <UNK><UNK><UNK><UNK><UNK>"#oulderoulderuccimbabwe pays"#" ����������"
    acc = evaluate_acc(formatted_prompt)
    if acc == 0:
        count += 1

In [30]:
count/1000

0.996

# positive-test

In [182]:
test_dataset_path_1 = './data/yelp-gpt2-control-only/raw/sentiment.test.1'
with open(test_dataset_path_1) as f: 
    sentences_test_1 = [line.strip() for line in f]

In [203]:
count = 0
for data in sentences_test_1[:2000]:
    formatted_prompt = data+" oulderoulderuccimbabwe pays"#" ����������"
    acc = evaluate_acc(formatted_prompt)
    if acc == 0:
        count += 1

In [204]:
count/2000

0.894

# SST-2

In [167]:
## Importing relevant libraries and dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from torch.autograd import Variable

from torchtext.legacy import data, datasets

import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

from numpy import linalg as LA
# import seaborn as sns; sns.set()

# Matplotlib plt default settings
plt.style.use('default')

In [168]:
## GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

## Hyperparameters
BATCH_SIZE = 1

cuda


In [169]:
## Download and processthe SST-2 corpus

# Set up input/output as torchtext Field objects
TEXT = data.Field(
    lower=True, # Lowercase all text
    batch_first=True) # order the tensor with batch as the first dimension  
LABEL = data.Field(
    sequential=False, # Do not tokenize the label (it is a single "pos"/"neg")
    unk_token=None) # Do not add an "unknown" token (not needed for label)

# Make splits for the dataset
# Filter out examples that are neutral
train_split, valid_split, test_split = datasets.SST.splits(
    TEXT, 
    LABEL, 
    filter_pred=lambda ex: ex.label != 'neutral')

# Build the vocabulary
# This builds an index such that "the" -> 1, "dog" -> 2 etc. 
TEXT.build_vocab(train_split,)
LABEL.build_vocab(train_split)

# Make iterator for splits
train_iter, valid_iter, test_iter = data.BucketIterator.splits(
    (train_split, valid_split, test_split), batch_size=BATCH_SIZE, device=device)

In [170]:
print('Number of unique words in our input texts')
print('len(TEXT.vocab)', len(TEXT.vocab))
print('Number of labels in our corpus')
print('len(LABEL.vocab)', len(LABEL.vocab))

Number of unique words in our input texts
len(TEXT.vocab) 14830
Number of labels in our corpus
len(LABEL.vocab) 2


In [171]:
list(train_split[:5])

In [112]:
# Get the first batch of the dataset
batch = next(iter(train_iter))

In [113]:
example_text = batch.text [0, :]
batch_label = batch.label.long()[0]

In [119]:
batch_label.item()

1

In [115]:
ind2sent (example_text).split("<pad>")[0]

'equilibrium the movie , as opposed to the manifesto , is really , really stupid .'

In [172]:
# Print the sentence given the indices
def ind2sent(vals):
    return (" ".join([TEXT.vocab.itos[i.item()] for i in vals]))

# Get the vectorized version of a sentence
def sent2ind(sentence):
    words = sentence.split (' ')
    arr = [TEXT.vocab.stoi[word] for word in words]
    return torch.tensor (arr).long()

In [33]:
ind2sent (example_text).split("<pad>")[0]

"it 's a big idea , but the film itself is small and shriveled . "

# train_iter - 2000

In [180]:
count = 0
x = 0
for batch in train_iter:
    x+= 1
    formatted_prompt = ind2sent(batch.text [0, :]).split("<pad>")[0]+" It was"#"����������"
    print(formatted_prompt)
    batch_label = batch.label.long()[0].item()
    acc = evaluate_acc(formatted_prompt)
    print('positive' if batch_label==0 else 'negative')
    if acc == batch_label:
        count += 1
    if x == 2000:
        break

been there , done that , liked it much better the first time around - when it was called the professional . It was
negative
it is hard not to be especially grateful for freedom after a film like this . It was
positive
the jokes are sophomoric , stereotypes are sprinkled everywhere and the acting ranges from bad to bodacious . It was
negative
none of this is half as moving as the filmmakers seem to think . It was
negative
this film is too busy hitting all of its assigned marks to take on any life of its own . It was
negative
too many scenarios in which the hero might have an opportunity to triumphantly sermonize , and too few that allow us to wonder for ourselves if things will turn out okay . It was
negative
ringu is a disaster of a story , full of holes and completely lacking in chills . It was
negative
possibly the most irresponsible picture ever released by a major film studio . It was
negative
wait to see it then . It was
negative
metaphors abound , but it is easy to take this film

positive
i suspect that you 'll be as bored watching morvern callar as the characters are in it . It was
negative
equilibrium the movie , as opposed to the manifesto , is really , really stupid . It was
negative
this breezy caper movie becomes a soulful , incisive meditation on the way we were , and the way we are . It was
positive
an unsophisticated sci-fi drama that takes itself all too seriously . It was
negative
... a rather bland affair . It was
negative
the chateau ... is less concerned with cultural and political issues than doting on its eccentric characters . It was
negative
a worthy addition to the cinematic canon , which , at last count , numbered 52 different versions . It was
positive
bigelow offers some flashy twists and turns that occasionally fortify this turgid fable . It was
positive
a lyrical metaphor for cultural and personal self-discovery and a picaresque view of a little-remembered world . It was
positive
the film gets close to the chimps the same way goodall did

positive
it takes you somewhere you 're not likely to have seen before , but beneath the exotic surface -lrb- and exotic dancing -rrb- it 's surprisingly old-fashioned . It was
positive
but one thing 's for sure : it never comes close to being either funny or scary . It was
negative
nothing wrong with performances here , but the whiney characters bugged me . It was
negative
like a grinning jack o ' lantern , its apparent glee is derived from a lobotomy , having had all its vital essence scooped out and discarded . It was
negative
a pretty decent kid-pleasing , tolerable-to-adults lark of a movie . It was
positive
this one is not nearly as dreadful as expected . It was
positive
it 's a terrible movie in every regard , and utterly painful to watch . It was
negative
there is no psychology here , and no real narrative logic -- just a series of carefully choreographed atrocities , which become strangely impersonal and abstract . It was
negative
that jack nicholson makes this man so watchabl

positive
kung pow is oedekerk 's realization of his childhood dream to be in a martial-arts flick , and proves that sometimes the dreams of youth should remain just that . It was
negative
sexy and romantic . It was
positive
the package in which this fascinating -- and timely -- content comes wrapped is disappointingly generic . It was
negative
inconsequential road-and-buddy pic . It was
negative
he makes you realize that deep inside righteousness can be found a tough beauty . It was
positive
lacks the spirit of the previous two , and makes all those jokes about hos and even more unmentionable subjects seem like mere splashing around in the muck . It was
negative
this is a children 's film in the truest sense . It was
positive
it 's rare to see a movie that takes such a speedy swan dive from `` promising '' to `` interesting '' to `` familiar '' before landing squarely on `` stupid '' . It was
negative
the art demands live viewing . It was
positive
contains a few big laughs but many mor

negative
swiftly deteriorates into a terribly obvious melodrama and rough-hewn vanity project for lead actress andie macdowell . It was
negative
it 's depressing to see how far herzog has fallen . It was
negative
... hokey art house pretension . It was
negative
a comedy that swings and jostles to the rhythms of life . It was
positive
good performances and a realistic , non-exploitive approach make paid in full worth seeing . It was
positive
one of those decades-spanning historical epics that strives to be intimate and socially encompassing but fails to do justice to either effort in three hours of screen time . It was
negative
most of the things that made the original men in black such a pleasure are still there . It was
positive
an uneven look into a grim future that does n't come close to the level of intelligence and visual splendour that can be seen in other films based on philip k. dick stories . It was
negative
the komediant is a tale worth catching . It was
positive
this is a mo

...

negative
... a hokey piece of nonsense that tries too hard to be emotional . It was
negative
femme fatale offers nothing more than a bait-and-switch that is beyond playing fair with the audience . It was
negative
the movie does such an excellent job of critiquing itself at every faltering half-step of its development that criticizing feels more like commiserating . It was
negative
... by the time it 's done with us , mira nair 's new movie has its audience giddy with the delight of discovery , of having been immersed in a foreign culture only to find that human nature is pretty much the same all over . It was
positive
there are n't many conclusive answers in the film , but there is an interesting story of pointed personalities , courage , tragedy and the little guys vs. the big guys . It was
positive
so mind-numbingly awful that you hope britney wo n't do it one more time , as far as movies are concerned . It was
negative
there are enough throwaway references to faith and rainbows to p

positive
evokes the 19th century with a subtlety that is an object lesson in period filmmaking . It was
positive
the screenplay is hugely overwritten , with tons and tons of dialogue -- most of it given to children . It was
negative
a gorgeously strange movie , heaven is deeply concerned with morality , but it refuses to spell things out for viewers . It was
positive
her performance moves between heartbreak and rebellion as she continually tries to accommodate to fit in and gain the unconditional love she seeks . It was
positive
five screenwriters are credited with the cliché-laden screenplay ; it seems as if each watered down the version of the one before . It was
negative
beautiful , angry and sad , with a curious sick poetry , as if the marquis de sade had gone in for pastel landscapes . It was
positive
a terrible movie that some people will nevertheless find moving . It was
negative
this little film is so slovenly done , so primitive in technique , that it ca n't really be called a

negative
cox creates a fluid and mesmerizing sequence of images to match the words of nijinsky 's diaries . It was
positive
a film that loses sight of its own story . It was
negative
winds up being both revelatory and narcissistic , achieving some honest insight into relationships that most high-concept films candy-coat with pat storylines , precious circumstances and beautiful stars . It was
positive
the first bond movie in ages that is n't fake fun . It was
positive
both deserve better . It was
negative
it 's about issues most adults have to face in marriage and i think that 's what i liked about it -- the real issues tucked between the silly and crude storyline . It was
positive
a weird and wonderful comedy . It was
positive
incoherence reigns . It was
negative
on its icy face , the new film is a subzero version of monsters , inc. , without the latter 's imagination , visual charm or texture . It was
negative
an off-beat and fanciful film about the human need for monsters to blame f

negative
this story of a determined woman 's courage to find her husband in a war zone offers winning performances and some effecting moments . It was
positive
the last kiss will probably never achieve the popularity of my big fat greek wedding , but its provocative central wedding sequence has far more impact . It was
positive
in questioning the election process , payami graphically illustrates the problems of fledgling democracies , but also the strength and sense of freedom the iranian people already possess , with or without access to the ballot box . It was
positive
it 's a feel-good movie about which you can actually feel good . It was
positive
cheap , vulgar dialogue and a plot that crawls along at a snail 's pace . It was
negative
isabelle huppert excels as the enigmatic mika and anna mouglalis is a stunning new young talent in one of chabrol 's most intense psychological mysteries . It was
positive
a story which fails to rise above its disgusting source material . It was
negat

negative


In [181]:
count/2000

0.533

# test_iter - all

In [205]:
count = 0
x = 0
for batch in test_iter:
    x+= 1
    formatted_prompt = ind2sent(batch.text [0, :]).split("<pad>")[0]+" oulderoulderuccimbabwe pays"#"����������"
    print(formatted_prompt)
    batch_label = batch.label.long()[0].item()
    acc = evaluate_acc(formatted_prompt)
    if acc == batch_label:
        count += 1
    

no. . oulderoulderuccimbabwe pays
ridiculous . oulderoulderuccimbabwe pays
good movie . oulderoulderuccimbabwe pays
good actress . oulderoulderuccimbabwe pays
brilliant ! ' oulderoulderuccimbabwe pays
no more . oulderoulderuccimbabwe pays
extremely dumb . oulderoulderuccimbabwe pays
extremely confusing . oulderoulderuccimbabwe pays
extremely boring . oulderoulderuccimbabwe pays
<unk> point . oulderoulderuccimbabwe pays
sorry , charlie oulderoulderuccimbabwe pays
i loved it ! oulderoulderuccimbabwe pays
is this <unk> ? oulderoulderuccimbabwe pays
it 's astonishing . oulderoulderuccimbabwe pays
<unk> to entertain ! oulderoulderuccimbabwe pays
a real clunker . oulderoulderuccimbabwe pays
what 's next ? oulderoulderuccimbabwe pays
bears is bad . oulderoulderuccimbabwe pays
it 's painful . oulderoulderuccimbabwe pays
i hate this movie oulderoulderuccimbabwe pays
an imaginative <unk> . oulderoulderuccimbabwe pays
a solidly <unk> chiller . oulderoulderuccimbabwe pays
a very funny movie . ould

lead <unk> testud and parmentier give superlative performances oulderoulderuccimbabwe pays
<unk> <unk> 's script is a gem . oulderoulderuccimbabwe pays
the new guy does have a heart . oulderoulderuccimbabwe pays
morton is , as usual , brilliant . oulderoulderuccimbabwe pays
clever , brutal and strangely soulful movie . oulderoulderuccimbabwe pays
an intriguing and entertaining introduction to <unk> . oulderoulderuccimbabwe pays
something the true film buff will enjoy . oulderoulderuccimbabwe pays
hey , who else needs a shower ? oulderoulderuccimbabwe pays
a slick , skillful little horror film . oulderoulderuccimbabwe pays
this is how you use special effects . oulderoulderuccimbabwe pays
notorious c.h.o. has oodles of vulgar <unk> . oulderoulderuccimbabwe pays
her film is unrelentingly claustrophobic and unpleasant . oulderoulderuccimbabwe pays
a <unk> , <unk> recipe for terror . oulderoulderuccimbabwe pays
the smartest <unk> comedy of the summer . oulderoulderuccimbabwe pays
but some u

now it 's a bad , <unk> movie . oulderoulderuccimbabwe pays
a trashy , exploitative , thoroughly unpleasant experience . oulderoulderuccimbabwe pays
another rent installment for the ian <unk> <unk> . oulderoulderuccimbabwe pays
the issue of faith is not explored very deeply oulderoulderuccimbabwe pays
men in black ii has <unk> something fierce . oulderoulderuccimbabwe pays
an ungainly , <unk> , b-movie rush job ... oulderoulderuccimbabwe pays
and forget about any attempt at a plot ! oulderoulderuccimbabwe pays
warmed-over tarantino by way of wannabe <unk> leonard . oulderoulderuccimbabwe pays
there 's not enough to sustain the comedy . oulderoulderuccimbabwe pays
a tv episode <unk> past its natural length . oulderoulderuccimbabwe pays
unfortunately , the picture failed to capture me . oulderoulderuccimbabwe pays
cliches are as thick as the <unk> smoke . oulderoulderuccimbabwe pays
the stories here suffer from the chosen format . oulderoulderuccimbabwe pays
fails to bring as much to the

majidi gets uniformly engaging performances from his largely amateur cast . oulderoulderuccimbabwe pays
jagger the actor is someone you want to see again . oulderoulderuccimbabwe pays
choose your reaction : <unk> -rrb- that sure is funny ! oulderoulderuccimbabwe pays
you 'll be more entertained getting hit by a <unk> . oulderoulderuccimbabwe pays
flat , but with a revelatory performance by michelle williams . oulderoulderuccimbabwe pays
... too slow , too boring , and occasionally annoying . oulderoulderuccimbabwe pays
the tone <unk> <unk> from tense to celebratory to <unk> . oulderoulderuccimbabwe pays
better at putting you to sleep than a sound machine . oulderoulderuccimbabwe pays
when the plot kicks in , the film loses credibility . oulderoulderuccimbabwe pays
the result is solemn and horrifying , yet strangely detached . oulderoulderuccimbabwe pays
the fight scenes are fun , but it grows tedious . oulderoulderuccimbabwe pays
-lrb- plays -rrb- in broad outline as pandering middle-a

a thoughtful look at a painful incident that made <unk> in 1995 . oulderoulderuccimbabwe pays
a triumph of art direction over narrative , but what art direction ! oulderoulderuccimbabwe pays
the movie has a soft , percolating magic , a deadpan suspense . oulderoulderuccimbabwe pays
the transporter is as lively and as fun as it is unapologetically dumb oulderoulderuccimbabwe pays
an amusing , <unk> <unk> documentary about life on the campaign trail . oulderoulderuccimbabwe pays
think of it as a sort of comfort food for the mind . oulderoulderuccimbabwe pays
compelling revenge thriller , though somewhat <unk> by a <unk> leading lady . oulderoulderuccimbabwe pays
a <unk> simple film , one that grows in power in <unk> . oulderoulderuccimbabwe pays
bullock does a good job here of working against her natural <unk> . oulderoulderuccimbabwe pays
what might have been a predictably heartwarming tale is <unk> with complexity . oulderoulderuccimbabwe pays
few films this year have been as <unk> in 

...

director kapur is a filmmaker with a real flair for epic landscapes and adventure , and this is a better film than his earlier english-language movie , the <unk> elizabeth . oulderoulderuccimbabwe pays
as <unk> as the special effects are , the folks who cobbled nemesis together indulge the force of humanity over hardware in a way that george lucas has long forgotten . oulderoulderuccimbabwe pays
watching <unk> and his gang put together his slasher video from spare parts and borrowed <unk> is as much fun as it must have been for them to make it . oulderoulderuccimbabwe pays
this insightful , oscar-nominated documentary , in which children on both sides of the <unk> conflict have their say away from <unk> parental eyes , gives peace yet another chance . oulderoulderuccimbabwe pays
moore 's complex and important film is also , believe it or not , immensely entertaining , a david and goliath story that 's still very much playing itself out . oulderoulderuccimbabwe pays
all the performances

... both hokey and <unk> , and definitely not in a hurry , so sit back , <unk> and have a few laughs while the little ones get a fuzzy treat . ' oulderoulderuccimbabwe pays
divine secrets of the ya-ya sisterhood may not be exactly divine , but it 's definitely -- defiantly -- <unk> <unk> , what with all of those terrific songs and spirited performances . oulderoulderuccimbabwe pays
time <unk> may not be the most memorable cinema <unk> but its profound <unk> message about our fragile existence and the absence of spiritual <unk> should at least <unk> an <unk> of <unk> oulderoulderuccimbabwe pays
as adapted by kevin <unk> from <unk> leys ' novel `` the death of napoleon '' and directed by alan taylor , napoleon 's journey is interesting but his <unk> <unk> is stillborn oulderoulderuccimbabwe pays
is n't quite the equal of woo 's best earlier work , but it 's easily his finest american film ... comes close to <unk> the brilliance of his hong kong films . oulderoulderuccimbabwe pays
althoug

it uses an old-time formula , it 's not terribly original and it 's rather messy -- but you just have to love the big , dumb , happy movie my big fat greek wedding . oulderoulderuccimbabwe pays
it 's rare to find a film that <unk> the eye , challenges the brain , and satisfies our lust for fast-paced action , but minority report delivers all that and a whole lot more . oulderoulderuccimbabwe pays
the whole cast looks to be having so much fun with the slapstick antics and silly street <unk> , <unk> around obscure expressions like <unk> and <unk> , that the <unk> 86 minutes <unk> by . oulderoulderuccimbabwe pays
this is as powerful a set of evidence as you 'll ever find of why art matters , and how it can resonate far beyond <unk> walls and through to the most painfully marginal lives . oulderoulderuccimbabwe pays
despite a story predictable enough to make the sound of music play like a <unk> thriller , its heart is so much in the right place it is difficult to get really <unk> at it . o

the makers of divine secrets of the ya-ya sisterhood should offer a free ticket -lrb- second <unk> , of course , two free tickets -rrb- to anyone who can <unk> a genuinely honest moment in their movie . oulderoulderuccimbabwe pays
although trying to balance <unk> humor and a normal ol' slasher plot seemed like a decent endeavor , the result does n't fully satisfy either the die-hard jason fans or those who can take a good joke . oulderoulderuccimbabwe pays
good for a few unintentional laughs , `` extreme ops '' was obviously made for the `` xxx '' crowd , people who enjoy mindless action without the benefit of decent acting , writing , and direction . oulderoulderuccimbabwe pays
cho 's fans are sure to be entertained ; it 's only fair in the interest of full <unk> to say that -- on the basis of this film alone -- i 'm not one of them . oulderoulderuccimbabwe pays
the overall feel of the film is pretty cheesy , but there 's still a real sense that the star trek tradition has been honore

though howard demonstrates a great eye as a director , this southern <unk> drama is sadly a tough sit , with an <unk> narrative and enough flashbacks and heavy-handed metaphors to choke a horse -- or at least slow him down to a <unk> . oulderoulderuccimbabwe pays
with a story inspired by the tumultuous surroundings of <unk> <unk> , where feelings of <unk> <unk> for every <unk> with a burst bubble , the <unk> has a few characters and ideas , but it never manages to put them on the same path . oulderoulderuccimbabwe pays
the film has a few cute ideas and several modest chuckles but it is n't exactly <unk> ... alas , santa is more <unk> than <unk> and the snowman -lrb- who never gets to play that <unk> -rrb- has all the charm of a <unk> . oulderoulderuccimbabwe pays
watching trouble every day , at least if you do n't know what 's coming , is like biting into what looks like a juicy , delicious <unk> on a hot summer day and coming away with your mouth full of rotten pulp and living <unk> .

In [206]:
count/len(test_iter)

0.7210323997803405

In [207]:
len(test_iter)

1821

# val_iter-2000

In [158]:
count = 0
x = 0
for batch in valid_iter:
    x+= 1
    formatted_prompt = ind2sent(batch.text [0, :]).split("<pad>")[0]+"����������"
    print(formatted_prompt)
    batch_label = batch.label.long()[0].item()
    acc = evaluate_acc(formatted_prompt)
    if acc == batch_label:
        count += 1
    if x == 2000:
        break

cool ?����������
bad .����������
very bad .����������
<unk> bleak and desperate����������
lovely and poignant .����������
oh come on .����������
yes , dull .����������
just not campy enough����������
funny but <unk> slight .����������
it 's fun <unk> .����������
one from the heart .����������
a delightful coming-of-age story .����������
part low rent <unk> .����������
slick piece of <unk> .����������
this movie is maddening .����������
a giggle a minute .����������
not exactly the <unk> knees����������
it 's just filler .����������
that 's a cheat .����������
hilariously inept and ridiculous .����������
fancy a real downer ?����������
a sometimes tedious film .����������
this is so bad .����������
or <unk> rat traps .����������
a deep and meaningful film .����������
a smart , witty follow-up .����������
people cinema at its finest .����������
<unk> moviemaking at its best .����������
<unk> ... revenge is sweet !����������
good old-fashioned <unk> is back !����������
it 's a beautiful m

the <unk> story needs more dramatic meat on its bones .����������
... is an <unk> attempt at directing by <unk> khouri .����������
i had to look away - this was god awful .����������
the cold turkey would 've been a far better title .����������
scores no points for originality , wit , or intelligence .����������
a sequel that 's much too big for its <unk> .����������
a string of <unk> sight gags based in insipid vulgarity .����������
plays like a volatile and overlong w <unk> fashion <unk> .����������
an occasionally funny , but overall limp , fish-out-of-water story .����������
burns never really <unk> to full effect the energetic cast .����������
an <unk> , would-be wacky , ultimately tedious sex farce .����������
the problem with this film is that it lacks focus .����������
but what are adults doing in the theater at all ?����������
<unk> <unk> every <unk> trick to give us the <unk> .����������
a sequence of ridiculous shoot - 'em - up scenes .����������
a beguiling splash of pastel

it offers little beyond the <unk> joys of pretty and weightless intellectual entertainment .����������
prurient <unk> aside , there 's little to love about this english trifle .����������
an infectious cultural fable with a tasty balance of family drama and frenetic comedy .����������
add yet another <unk> to a talented head , clooney 's a good director .����������
ultimately feels empty and unsatisfying , like <unk> a <unk> <unk> without the <unk> .����������
a <unk> tale of love , betrayal , revenge and above all , faith .����������
for the most part stevens glides through on some solid performances and witty dialogue .����������
against all odds in heaven and hell , it <unk> me out just fine .����������
the second coming of harry potter is a film far superior to its predecessor .����������
manages to transcend the sex , drugs and <unk> plot into something far richer .����������
a study in shades of <unk> , offering itself up in subtle plot maneuvers ...����������
for movie lovers as

manages to show life in all of its banality when the <unk> is quite the opposite .����������
tries to add some spice to its quirky sentiments but the taste is all too familiar .����������
do n't be <unk> by the impressive cast list - eye see you is pure junk .����������
and when you 're talking about a slapstick comedy , that 's a pretty big problem .����������
director uwe boll and the actors provide scant reason to care in this crude '70s throwback .����������
this is a shameless sham , calculated to cash in on the popularity of its stars .����������
there ought to be a directing license , so that ed burns can have his <unk> .����������
a lackluster , <unk> sequel to the classic disney adaptation of <unk> barrie 's peter pan .����������
adults will wish the movie were less simplistic , obvious , clumsily plotted and <unk> characterized .����������
every time you look , sweet home alabama is taking another <unk> of a wrong turn .����������
the only excitement comes when the credits fi

the acting , costumes , music , cinematography and sound are all astounding given the production 's <unk> locales .����������
reign of fire looks as if it was made without much thought -- and is best watched that way .����������
you do n't have to know about music to appreciate the film 's <unk> blend of comedy and romance .����������
in all , this is a watchable movie that 's not quite the memorable experience it might have been .����������
it is great summer fun to watch arnold and his buddy <unk> <unk> off a quirky cast of characters .����������
allows us to hope that nolan is <unk> to <unk> a major career as a commercial yet inventive filmmaker .����������
macdowell , whose <unk> southern charm has anchored lighter <unk> ... brings an absolutely riveting conviction to her role .����������
a gorgeous , high-spirited musical from india that exquisitely blends music , dance , song , and high drama .����������
the vitality of the actors keeps the intensity of the film high , even as th

...

the experience of going to a film festival is a rewarding one ; the <unk> of <unk> one through this movie is not .����������
the movie is what happens when you blow up small <unk> to 10 times their natural size , and it ai n't pretty .����������
a dumb movie with dumb characters doing dumb things and you have to be really dumb not to see where this is going .����������
for this reason and this reason only -- the power of its own steadfast , <unk> convictions -- chelsea walls deserves a <unk> .����������
... turns so <unk> trite in its last 10 minutes that anyone without a <unk> sweet tooth will likely go into sugar shock .����������
sustains its dreamlike <unk> through a <unk> of cheesy <unk> and <unk> cheap effects , not the least of which is <unk> <unk> .����������
it can not be enjoyed , even on the level that one enjoys a bad slasher flick , primarily because it is dull .����������
green might want to hang onto that <unk> mask , as <unk> may be the only way to pay for his next proj

a compelling spanish film about the <unk> effects of jealousy in the life of a young <unk> whose sexual passion for her husband becomes an obsession .����������
the format gets used best ... to capture the <unk> heights achieved by <unk> and bmx <unk> , whose <unk> <unk> occasionally ends in <unk> <unk> .����������
for the first time in years , de niro digs deep emotionally , perhaps because he 's been <unk> by the powerful work of his co-stars .����������
the movie is n't just hilarious : it 's witty and inventive , too , and in <unk> , it is n't even all that dumb .����������
candid and comfortable ; a film that deftly balances action and reflection as it lets you grasp and feel the passion others have for their work .����������
my wife is an actress is an utterly charming french comedy that feels so american in sensibility and style it 's virtually its own hollywood remake .����������
few films capture so perfectly the hopes and dreams of little boys on baseball fields as well as th

an effectively creepy , <unk> -lrb- not <unk> -rrb- film from japanese director <unk> nakata , who takes the <unk> curse on chain <unk> and actually applies it .����������
having had the good sense to cast actors who are , generally speaking , <unk> by the movie-going public , khouri then gets terrific performances from them all .����������
-lrb- chaiken 's -rrb- talent lies in an evocative , accurate observation of a distinctive milieu and in the lively , convincing dialogue she creates for her characters .����������
two hours fly by -- opera 's a pleasure when you do n't have to endure <unk> -- and even a <unk> to the form comes away <unk> .����������
it <unk> you , you ca n't forget it , you admire its conception and are able to <unk> some of the <unk> you had while watching it .����������
<unk> 's naturalistic performance speaks volumes more truth than any ` reality ' show , and anybody <unk> their own drastic life changes should watch some body first .����������
an exhilarating fu

cq 's reflection of artists and the love of <unk> suggests nothing less than a new voice that deserves to be considered as a possible successor to the best european directors .����������
passable entertainment , but it 's the kind of motion picture that wo n't make much of a splash when it 's released , and will not be remembered long afterwards .����������
belongs to daniel day-lewis as much as it belongs to martin scorsese ; it 's a memorable performance in a big , <unk> , disturbing , unusual and highly successful film .����������
pumpkin takes an admirable look at the hypocrisy of political correctness , but it does so with such an uneven tone that you never know when humor ends and tragedy begins .����������
the quality of the art combined with the humor and intelligence of the script allow the filmmakers to present the <unk> message of forgiveness without it ever becoming preachy or <unk> .����������
another one of those <unk> <unk> movies like `` divine secrets of the <unk> <unk

it 's somewhat clumsy and too lethargically paced -- but its story about a mysterious creature with psychic abilities offers a solid build-up , a terrific climax , and some nice chills along the way .����������
an interesting story with a pertinent -lrb- cinematically unique -rrb- message , told fairly well and scored to perfection , i found myself struggling to put my finger on that elusive `` missing thing . ''����������
it does nothing new with the old story , except to show <unk> in this sort of <unk> slow motion that makes the gang <unk> look like they 're being streamed over a <unk> modem .����������
the tale of <unk> -lrb- <unk> lau -rrb- , a sleek <unk> on the trail of o -lrb- takashi <unk> -rrb- , the most legendary of asian <unk> , is too <unk> to take hold .����������
samira makhmalbaf 's new film <unk> is much like the ethos of a stream of consciousness , although , it 's unfortunate for the viewer that the thoughts and <unk> coming through are <unk> and banal����������
not

In [159]:
count/2000

0.3115